# Domain-Specific RAG: A Practical Tutorial



## Learning Objectives

By the end of this tutorial, you will be able to:
 - Understand the basic concepts of Retrieval-Augmented Generation (RAG).
 - Implement a simple RAG pipeline using pre-built indexes and models.
 - Apply domain-specific prompting and evaluation metrics to tailor RAG to different use cases.
 - Explore the impact of different retrieval and generation parameters on the performance of a RAG system.

## Exercises and Challenges (Optional)

 - **Experiment with different datasets:** Try using other pre-built indexes or creating your own indexes from a dataset of your choice.
 - **Fine-tune the retrieval model:** Explore fine-tuning the bi-encoder or cross-encoder to improve retrieval accuracy for your domain.
 - **Evaluate different LLMs:** Compare the performance of different LLMs for answer generation in your use case.
 - **Build a user interface:** Develop a simple web application or chatbot that integrates your RAG pipeline.

## 🖥️ System Requirements & Setup

### System Requirements

This tutorial uses deep learning models and vector similarity search that benefit from:

- **Compute Resources:**
  - **RAM:** Minimum 8GB, recommended 16GB+
  - **GPU:** Recommended for faster processing (especially when using cross-encoders)
  - **Storage:** ~2GB for pre-built indexes and models
  - **Google Colab:** This notebook runs well on a free Colab instance with T4 GPU

### 🧰 Tools & Libraries We'll Use

This tutorial leverages several key libraries:

- **[sentence-transformers](https://www.sbert.net/)**: For embedding generation and cross-encoding
- **[FAISS](https://github.com/facebookresearch/faiss)**: For efficient similarity search
- **[ir_datasets](https://ir-datasets.com/)**: For accessing benchmark datasets
- **[Mistral AI](https://mistral.ai/)**: For answer generation using LLMs
- **[Hugging Face Hub](https://huggingface.co/docs/hub/index)**: For accessing pre-built indexes

### 🔑 Mistral API Setup

To use the Mistral AI API for answer generation:

1. Create a Mistral AI account at [https://console.mistral.ai/](https://console.mistral.ai/)
2. Generate an API key from your account dashboard
3. Add your key to this notebook:
   ```python
   os.environ["MISTRAL_API_KEY"] = "YOUR_API_KEY_HERE"  # Replace with your actual key
   ```

## 🏗️ RAG Architecture Overview

Retrieval-Augmented Generation (RAG) enhances Large Language Models (LLMs) by retrieving relevant information from external knowledge sources before generating responses. This approach significantly improves accuracy and allows LLMs to access domain-specific information without retraining.

![RAG Architecture Overview](https://drive.google.com/uc?export=view&id=1U28sDU5JAE-UfIfT9-BZ_4FEHktpuiah)

### Core Components of RAG:

1. **Document Processing Pipeline**
   - **Collection**: Gathering domain-specific documents, articles, or knowledge bases
   - **Chunking**: Breaking documents into manageable segments (paragraphs or sections)
   - **Embedding**: Converting text chunks into dense vector representations

2. **Retrieval System**
   - **Query Processing**: Converting user questions into the same vector space
   - **Vector Search**: Finding relevant document chunks using similarity measures
   - **Reranking**: Further refining results using more sophisticated models

3. **Generation System**
   - **Context Assembly**: Combining retrieved information into a prompt
   - **LLM Integration**: Sending the enriched prompt to an LLM
   - **Response Generation**: Creating coherent, accurate, and contextual answers

### Why Domain-Specific RAG Matters

Different domains require specialized knowledge and specific response formats:
- Medical advice needs evidence-based information and appropriate caveats
- Technical support requires precise, actionable instructions
- Educational content should be structured for different learning levels

In this tutorial, we'll customize each component of the RAG pipeline for specific domains.

### 🧩 Domain-Specific RAG Architecture

Standard RAG pipelines can be enhanced by domain-specific customizations at each stage:

### Domain Customization Points:

1. **Data Sources & Preprocessing**
   - Scientific: Research papers, clinical trials, structured abstracts
   - Technical: Documentation, forums, code repositories
   - Educational: Textbooks, lecture notes, graded materials

2. **Embedding & Retrieval**
   - Domain-specific models (e.g., BioBERT for medical, CodeBERT for programming)
   - Customized chunking strategies (e.g., by sections, by concepts)
   - Specialized ranking functions (e.g., recency weighting for news)

3. **Prompt Engineering**
   - Domain-appropriate instructions, terminology, and formats
   - Specialty-specific constraints and guardrails
   - Role-specific personas (e.g., educator, researcher, consultant)

4. **Evaluation Metrics**
   - Scientific: Citation accuracy, evidence quality
   - Technical: Step completeness, actionability
   - Educational: Clarity at appropriate learning level

In this tutorial, we'll focus on practical implementations of these customizations.

## 📚 Datasets

This tutorial uses a diverse set of datasets from the [BEIR benchmark collection](https://github.com/beir-cellar/beir) (via `ir_datasets`) to demonstrate how Retrieval-Augmented Generation (RAG) can be applied across multiple real-world domains.

These datasets represent realistic information needs from different user groups, such as researchers, developers, educators, and the general public. Pre-built indexes have been provided for each dataset to ensure the tutorial runs smoothly in a short time window (~30 minutes), avoiding the overhead of on-the-fly index construction.

### 🧪 Scientific Research Domain
- `beir/trec-covid`: Focused on COVID-19 research questions using the CORD-19 corpus.
- `beir/scifact`: Scientific claim verification, where the system must support or refute claims using scientific abstracts.
- `beir/nfcorpus`: Non-factoid biomedical QA, based on user questions from the NLM’s PubMed Helpdesk.

### 🛠️ Technical Support Domain
- `beir/cqadupstack/android`: Community Q&A data from Stack Exchange on Android development.
- `beir/cqadupstack/webmasters`: Web hosting and webmaster technical queries.
- `beir/cqadupstack/unix`: Unix/Linux command-line and scripting support.

### 🎓 Education & Library Domain
- `beir/natural-questions`: Real user questions from Google Search and answers from Wikipedia.
- `beir/hotpotqa`: Multi-hop QA dataset requiring reasoning over multiple Wikipedia documents.
- `beir/nfcorpus`: Also used here for medically-themed educational queries.

### 🔍 Fact Verification Domain
- `beir/fever`: Fact-checking based on Wikipedia claims.
- `beir/climate-fever`: Focused on climate change-related claims and evidence.
- `beir/scifact`: Also shared here for scientific claim verification.

### 🏥 Healthcare Information Domain
- `beir/nfcorpus`: Used for health-related literature review and question answering.
- `beir/trec-covid`: Reused here to address health policy and treatment questions during the pandemic.

> ℹ️ Each domain has one **default dataset** selected (the first in each list), but you can explore other datasets in that domain using the dropdown selector. All datasets are accessed through the `ir_datasets` library, and prebuilt indexes are provided to ensure quick experimentation.

## Preliminaries

Install required packages



In [ ]:
!pip install -q sentence-transformers transformers torch numpy faiss-cpu tqdm ir_datasets ir_measures pandas matplotlib ipywidgets huggingface_hub

Import required libraries



In [ ]:
import os
import torch
import requests
import json
import numpy as np
import time
import pandas as pd
from tqdm.notebook import tqdm
import ir_datasets
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from transformers import AutoTokenizer, AutoModelForCausalLM
import faiss
import pickle
import ir_measures
from ir_measures import *
import matplotlib.pyplot as plt
from IPython.display import display, HTML, Markdown
import ipywidgets as widgets
from IPython.display import display, clear_output
from huggingface_hub import hf_hub_download

Set up GPU/CPU device

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
HUB_REPO_ID = "ShubhamC/rag-tutorial-prebuilt-indexes"

In [ ]:
import warnings
warnings.filterwarnings("ignore")

## 🔍 Basic RAG Pipeline: A Minimal Example

Before diving into domain customization, let's understand the basic RAG workflow with a minimal working example. This gives us a foundation to build upon.

### The Four Essential Steps:

1. Load a document collection
2. Convert user query to a vector
3. Retrieve relevant documents
4. Generate an answer using retrieved information

In [ ]:
# Quick demonstration of a minimal RAG pipeline
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# 1. Create a tiny document collection (normally this would be loaded from a database)
mini_docs = [
    "RAG stands for Retrieval-Augmented Generation in AI systems.",
    "Embedding models convert text into numerical vectors.",
    "FAISS is a library for efficient similarity search in vector spaces.",
    "Language models generate text based on provided prompts."
]

print("1. Loaded document collection with", len(mini_docs), "documents\n")

# 2. Create embeddings for documents
mini_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
doc_embeddings = mini_model.encode(mini_docs)
print("2. Created document embeddings with shape:", doc_embeddings.shape, "\n")

# Create a simple vector index
dimension = doc_embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(doc_embeddings)
print("3. Built search index with", index.ntotal, "vectors\n")

# 3. Process a query and retrieve
query = "What is RAG in AI?"
query_vector = mini_model.encode([query])
D, I = index.search(query_vector, k=2)  # Get top 2 results

print("Query:", query)
print("\nRetrieved documents:")
for i, doc_idx in enumerate(I[0]):
    print(f"[{i+1}] {mini_docs[doc_idx]} (distance: {D[0][i]:.2f})")

# 4. Generate answer (simplified simulation without actual LLM API call)
print("\n4. Generated Answer:")
print(f"RAG (Retrieval-Augmented Generation) is an AI technique that enhances language models by retrieving relevant information before generating responses. It combines the knowledge retrieval capabilities of search systems with the text generation abilities of large language models.")

## About Pre-built Indexes

For this tutorial, we're using pre-built indexes to save time. The indexes were created in advance
using datasets from various sources:

- BEIR (Benchmarking IR): Contains various IR tasks including scientific literature, news articles, etc.
- Stack Exchange collections: Technical Q&A across domains
- CORD-19: COVID-19 related research papers
- NQ (Natural Questions): General knowledge Q&A

The pre-built indexes include:
1. Document corpus with text and titles
2. Document embeddings using sentence-transformers
3. FAISS index for efficient retrieval
4. Sample queries with relevance judgments

In a typical RAG pipeline, you would need to:
1. Download and process a dataset
2. Create document embeddings
3. Build a search index
4. Perform retrieval
5. Generate answers with LLM

For this tutorial, steps 1-3 are already done for you with the pre-built indexes.
We'll focus on steps 4-5: retrieval and generation.

## 🎯 Use Cases in This Tutorial

This tutorial explores how Retrieval-Augmented Generation (RAG) can be customized for different real-world domains. We've defined five practical **use cases**, each mapped to datasets available in the [`ir_datasets`](https://ir-datasets.com/) library and paired with domain-specific example queries and prompts.

Each use case provides:
- A curated set of relevant datasets
- Example queries based on real-world information needs
- A system prompt that guides the language model’s tone and behavior

These use cases demonstrate how RAG can go beyond generic QA to serve **domain-specific goals** like academic research, troubleshooting, or fact verification.

---

### 🧪 1. Scientific Research
Designed for tasks like literature review, scientific explanation, and research comprehension.

- **Default Dataset:** `beir/trec-covid`
- **Other Datasets:** `beir/scifact`, `beir/nfcorpus`
- **Example Queries:**
  - What are the most effective treatments for severe COVID-19?
  - How does mRNA vaccine technology work?
- **Prompt:** _You are a scientific research assistant. Provide accurate, evidence-based answers with appropriate scientific context and caveats._

---

### 🛠️ 2. Technical Support
Targets IT and developer support tasks using Stack Exchange-style technical Q&A data.

- **Default Dataset:** `beir/cqadupstack/android`
- **Other Datasets:** `beir/cqadupstack/webmasters`, `beir/cqadupstack/unix`
- **Example Queries:**
  - How do I fix network connectivity issues with Android devices?
  - What's causing my app to crash on startup?
- **Prompt:** _You are a technical support specialist. Provide clear, step-by-step solutions to technical problems with practical troubleshooting advice._

---

### 🎓 3. Education & Library
Supports research help, study guides, and educational content generation.

- **Default Dataset:** `beir/natural-questions`
- **Other Datasets:** `beir/hotpotqa`, `beir/nfcorpus`
- **Example Queries:**
  - What teaching methods are most effective for student engagement?
  - How did World War II impact post-war economic development?
- **Prompt:** _You are an educational assistant. Provide informative, well-structured answers suitable for learners, with clear explanations of complex concepts._

---

### 🔍 4. Fact Verification
Focuses on verifying potentially controversial or widely debated claims using factual sources.

- **Default Dataset:** `beir/fever`
- **Other Datasets:** `beir/scifact`, `beir/climate-fever`
- **Example Queries:**
  - Do vaccines cause autism?
  - Is 5G technology harmful to human health?
- **Prompt:** _You are a fact-checking assistant. Provide balanced, evidence-based assessments of claims with references to sources where possible._

---

### 🏥 5. Healthcare Information
Geared toward health and medical information, patient education, and clinical understanding.

- **Default Dataset:** `beir/nfcorpus`
- **Other Datasets:** `beir/trec-covid`
- **Example Queries:**
  - What are the potential side effects of statin medications?
  - How effective is cognitive behavioral therapy for anxiety?
- **Prompt:** _You are a healthcare information assistant. Provide evidence-based answers about medical topics, while noting that this is not medical advice. Focus on established research and clinical guidelines._


## Setup


Select RAG Use Case

In [ ]:
# Dictionary of use cases with their descriptions, datasets, and example queries
# Focus on datasets publicly available in ir_datasets but using prebuilt indexes
use_cases = {
    "Scientific Research": {
        "description": "Support for scientific literature review, fact verification, and research paper comprehension",
        "datasets": ["beir/trec-covid", "beir/scifact", "beir/nfcorpus"],
        "default_dataset": "beir/trec-covid",
        "example_queries": [
            "What are the most effective treatments for severe COVID-19?",
            "How does mRNA vaccine technology work?",
            "What evidence supports aerosol transmission of respiratory viruses?",
            "What is the relationship between diet and cancer prevention?"
        ],
        "domain_prompt": "You are a scientific research assistant. Provide a clear, accurate, and evidence-based answer to the user's question using only the retrieved documents. Cite supporting information from the context explicitly, and include appropriate scientific context, limitations, and caveats where applicable. Do not speculate beyond the provided material."

    },
    "Technical Support": {
        "description": "IT helpdesk, programming assistance, and technical knowledge base",
        "datasets": ["beir/cqadupstack/android", "beir/cqadupstack/webmasters", "beir/cqadupstack/unix"],
        "default_dataset": "beir/cqadupstack/android",
        "example_queries": [
            "How do I fix network connectivity issues with Android devices?",
            "What's causing my app to crash on startup?",
            "How to implement pagination in a mobile application?",
            "Best practices for securing an Android device"
        ],
        "domain_prompt": "You are a technical support specialist. Based on the retrieved documents, provide a concise and practical solution to the user's problem. Structure your answer step-by-step, cite relevant technical details, and ensure instructions are clear and executable. Avoid speculation or unsupported suggestions."

    },
    "Education & Library": {
        "description": "Enhanced research assistance, study materials, and educational content",
        "datasets": ["beir/nfcorpus", "beir/natural-questions", "beir/hotpotqa"],
        "default_dataset": "beir/natural-questions",
        "example_queries": [
            "What teaching methods are most effective for student engagement?",
            "How did World War II impact economic development in post-war Europe?",
            "What are the key differences between behaviorist and constructivist learning theories?",
            "How does cellular respiration relate to photosynthesis?"
        ],
        "domain_prompt": "You are an educational assistant. Using the retrieved documents, provide a well-structured, accurate, and learner-friendly explanation of the topic. Break down complex concepts into simpler terms, include definitions or examples when needed, and ensure your tone is clear, supportive, and informative. Ground your answer entirely in the context provided."

    },
    "Fact Verification": {
        "description": "Verifying claims and statements across various sources",
        "datasets": ["beir/fever", "beir/scifact", "beir/climate-fever"],
        "default_dataset": "beir/fever",
        "example_queries": [
            "Is climate change primarily caused by human activities?",
            "Do vaccines cause autism?",
            "Does vitamin C prevent the common cold?",
            "Is 5G technology harmful to human health?"
        ],
        "domain_prompt": "You are a fact-checking assistant. Using only the retrieved documents, assess the accuracy of the user's claim. Provide a balanced, evidence-based analysis with references to specific statements or sources from the documents. If the evidence is inconclusive, clearly state the uncertainty and avoid speculation."

    },
    "Healthcare Information": {
        "description": "Medical literature review, patient education, and clinical guidelines",
        "datasets": ["beir/nfcorpus", "beir/trec-covid"],
        "default_dataset": "beir/nfcorpus",
        "example_queries": [
            "What is the relationship between diet and heart disease?",
            "How effective is cognitive behavioral therapy for anxiety disorders?",
            "What are the potential side effects of statin medications?",
            "How does family history affect cancer risk assessment?"
        ],
        "domain_prompt": "You are a healthcare information assistant. Based on the retrieved documents, provide an accurate, evidence-based summary addressing the user's question. Do not offer medical advice. Instead, focus on established research findings, clinical guidelines, and clearly state any risks, limitations, or uncertainties in the available evidence. Ground your answer entirely in the context provided."
    },
    "Campus Info": {
        "description": "Ask questions about services, offices, and procedures listed on the S&T website.",
        "datasets": ["custom_mst_site"],
        "default_dataset": "custom_mst_site",
        "example_queries": [
            "Where is the ISSS office located?",
            "What services are offered to sponsored international students?",
            "How can I contact the Graduate Studies department?",
            "Where can I find the event calendar?"
        ],
        "domain_prompt": "You are a helpful campus assistant for Missouri S&T. Using only the information available in the retrieved content, answer the user's question clearly and accurately. Reference the exact page or section when possible. Be concise, avoid speculation, and maintain a friendly and professional tone."

    }
}

In [ ]:
# Step 1: Create dropdowns
use_case_dropdown = widgets.Dropdown(
    options=list(use_cases.keys()),
    description='Use Case:',
    value="Scientific Research"
)

dataset_dropdown = widgets.Dropdown(
    options=use_cases["Scientific Research"]["datasets"],
    description='Dataset:'
)

# Step 2: Update dataset list when use case changes
def update_datasets(change):
    new_use_case = change['new']
    new_datasets = use_cases[new_use_case]["datasets"]
    dataset_dropdown.options = new_datasets
    dataset_dropdown.value = use_cases[new_use_case]["default_dataset"]

use_case_dropdown.observe(update_datasets, names='value')

# Step 3: Display both dropdowns
display(use_case_dropdown, dataset_dropdown)

# Step 4: Function to confirm selections and assign variables
def confirm_selection(_):
    global use_case, selected_dataset
    use_case = use_case_dropdown.value
    selected_dataset = dataset_dropdown.value

    clear_output()
    print(f"✅ Use Case Selected: {use_case}")
    print(f"✅ Dataset Selected: {selected_dataset}")
    print(f"\nDescription:\n{use_cases[use_case]['description']}")
    print("\nExample queries:")
    for query in use_cases[use_case]['example_queries']:
        print(f"- {query}")

# Step 6: Confirm button
confirm_button = widgets.Button(description="Confirm Selection", button_style='success')
confirm_button.on_click(confirm_selection)
display(confirm_button)

Define the paths where pre-built index files should be

In [ ]:
base_path = f"prebuilt_indexes/{selected_dataset.replace('/', '_')}"
corpus_path = f"{base_path}/corpus.pkl"
embeddings_path = f"{base_path}/embeddings.npy"
faiss_index_path = f"{base_path}/faiss_index.bin"
doc_ids_path = f"{base_path}/doc_ids.pkl"

Create the directory if it doesn't exist


In [ ]:
os.makedirs(base_path, exist_ok=True)

If you need to delete the directory structure for some reason, you can use the code below. Otherwise ignore.

In [ ]:
import os
import shutil

def empty_directory(directory_path):
    """Remove all files and subdirectories within a directory without deleting the directory itself."""
    try:
        # Check if the directory exists
        if not os.path.exists(directory_path):
            print(f"⚠️ Directory doesn't exist: {directory_path}")
            return False

        # List all items in the directory
        for item in os.listdir(directory_path):
            item_path = os.path.join(directory_path, item)

            # If it's a file, delete it
            if os.path.isfile(item_path) or os.path.islink(item_path):
                os.unlink(item_path)
                print(f"  🗑️ Deleted file: {item}")

            # If it's a directory, delete it and its contents
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
                print(f"  🗑️ Deleted subdirectory: {item}")

        print(f"✅ Successfully emptied directory: {directory_path}")
        return True
    except Exception as e:
        print(f"❌ Error emptying directory {directory_path}: {e}")
        return False

def delete_directory_safely(directory_path):
    """Empty a directory and then delete it."""
    if empty_directory(directory_path):
        try:
            os.rmdir(directory_path)
            print(f"✅ Successfully deleted directory: {directory_path}")
            return True
        except OSError as e:
            print(f"❌ Error deleting directory {directory_path}: {e}")
            return False
    return False

# Example usage:
nested_dir = "prebuilt_indexes/custom_mst_site/custom_mst_site"

# Option 1: Empty the directory first, then delete it
# empty_directory(nested_dir)
# os.rmdir(nested_dir)

# OR Option 2: Do both operations in one function call
delete_directory_safely(base_path)

Function to generate sample corpus based on the selected dataset and use case

In [ ]:
def generate_sample_corpus(dataset_name, current_use_case):
    """Generate a sample corpus with domain-specific content based on the dataset name."""
    corpus = {}

    if current_use_case == "Scientific Research":
        # Scientific research content
        for i in range(20):
            doc_id = f"doc{i}"
            if "covid" in dataset_name.lower():
                corpus[doc_id] = {
                    "title": f"COVID-19 Research Paper {i}",
                    "text": f"This paper investigates the effects of COVID-19 on respiratory health. We conducted a study with {100+i} patients and found significant correlations between viral load and symptom severity. The study suggests that early intervention with antiviral medications may reduce hospitalization rates."
                }
            elif "scifact" in dataset_name.lower():
                corpus[doc_id] = {
                    "title": f"Scientific Study on Topic {i}",
                    "text": f"Our research demonstrates that hypothesis {i} is supported by experimental evidence. The data shows a statistically significant effect (p<0.05) across multiple trials. These findings contradict previous assumptions and suggest a new mechanism for this phenomenon."
                }
            else:
                corpus[doc_id] = {
                    "title": f"Scientific Paper {i}",
                    "text": f"This research examines the relationship between diet and health outcomes. Analysis of data from {500+i} participants shows significant associations between consumption of processed foods and increased risk of chronic diseases. The findings highlight the importance of dietary interventions in public health strategies."
                }

    elif current_use_case == "Technical Support":
        # Technical support content
        for i in range(20):
            doc_id = f"doc{i}"
            if "android" in dataset_name.lower():
                corpus[doc_id] = {
                    "title": f"Android Technical Issue {i}",
                    "text": f"Users experiencing battery drain on Android devices should check for apps running in the background. The issue often occurs after system updates or when location services are constantly active. To fix this, go to Settings > Battery > Battery Usage and identify power-hungry applications. Restricting background activity for these apps can significantly improve battery life."
                }
            elif "webmasters" in dataset_name.lower():
                corpus[doc_id] = {
                    "title": f"Web Development Problem {i}",
                    "text": f"When implementing responsive designs, developers often encounter issues with viewport rendering on mobile devices. To address this, ensure your HTML includes the proper meta viewport tag. Also check that media queries are correctly implemented to handle different screen sizes. Testing across multiple devices is essential to verify responsive behavior."
                }
            else:
                corpus[doc_id] = {
                    "title": f"Technical Solution {i}",
                    "text": f"A common error when setting up network connections is incorrect DNS configuration. To troubleshoot, first verify that the DNS server addresses are correct. Then flush the DNS cache to ensure old records aren't causing conflicts. If problems persist, try using alternative DNS servers to determine if the issue is with your ISP's DNS resolution."
                }

    elif current_use_case == "Education & Library":
        # Educational content
        for i in range(20):
            doc_id = f"doc{i}"
            if "questions" in dataset_name.lower():
                corpus[doc_id] = {
                    "title": f"Educational Topic {i}",
                    "text": f"This article explains the fundamental concepts of learning theories. Constructivism emphasizes how learners actively build knowledge through experience and reflection, while behaviorism focuses on observable behaviors and environmental conditioning. Understanding these frameworks helps educators design more effective teaching strategies that accommodate different learning styles and cognitive processes."
                }
            elif "hotpot" in dataset_name.lower():
                corpus[doc_id] = {
                    "title": f"Historical Event {i}",
                    "text": f"The Industrial Revolution transformed economic systems through mechanization and factory production. Beginning in Britain in the late 18th century, it spread throughout Europe and North America, fundamentally changing social structures and labor practices. Technological innovations like the steam engine drove unprecedented growth while creating new challenges related to urbanization, working conditions, and economic inequality."
                }
            else:
                corpus[doc_id] = {
                    "title": f"Educational Resource {i}",
                    "text": f"This educational material covers key scientific concepts for secondary education. Topics include cellular biology, chemical reactions, and physical laws. The content is structured to build conceptual understanding through progressive learning objectives, providing examples and applications relevant to students' daily experiences."
                }

    elif current_use_case == "Fact Verification":
        # Fact verification content
        for i in range(20):
            doc_id = f"doc{i}"
            if "fever" in dataset_name.lower():
                corpus[doc_id] = {
                    "title": f"Fact Check Article {i}",
                    "text": f"Climate scientists have reached overwhelming consensus that human activities are the primary driver of observed climate change since the mid-20th century. Multiple independent lines of evidence support this conclusion, including atmospheric CO2 measurements, temperature records, and climate model projections. Natural factors alone cannot explain the rapid warming observed in recent decades."
                }
            elif "scifact" in dataset_name.lower():
                corpus[doc_id] = {
                    "title": f"Scientific Claim Assessment {i}",
                    "text": f"Research has consistently failed to find evidence supporting a causal link between vaccines and autism. Multiple large-scale epidemiological studies involving millions of children have found no association between vaccination and autism spectrum disorders. The original study suggesting this link was retracted due to methodological flaws and ethical violations."
                }
            else:
                corpus[doc_id] = {
                    "title": f"Fact Verification {i}",
                    "text": f"Analysis of the claim that 5G technology poses health risks finds insufficient scientific evidence to support this assertion. While all radiation sources deserve study, 5G operates using non-ionizing radiation that lacks sufficient energy to damage cellular DNA. Current research indicates that exposure levels from 5G infrastructure fall well below international safety guidelines."
                }

    elif current_use_case == "Healthcare Information":
        # Healthcare content
        for i in range(20):
            doc_id = f"doc{i}"
            if "nfcorpus" in dataset_name.lower():
                corpus[doc_id] = {
                    "title": f"Nutrition Research Study {i}",
                    "text": f"This review examines the relationship between dietary patterns and cardiovascular health. Evidence consistently shows that diets rich in fruits, vegetables, whole grains, and lean proteins are associated with reduced risk of heart disease. Conversely, high consumption of processed foods, saturated fats, and added sugars correlates with increased cardiovascular risk factors including hypertension and elevated cholesterol."
                }
            elif "covid" in dataset_name.lower():
                corpus[doc_id] = {
                    "title": f"COVID-19 Treatment Review {i}",
                    "text": f"Clinical trials evaluating antiviral medications for COVID-19 have shown varying degrees of efficacy. Early treatment with certain antivirals may reduce symptom duration and hospitalization risk in high-risk populations. However, treatment effectiveness depends on timing, viral variants, and patient characteristics. Comprehensive management approaches typically include supportive care alongside targeted therapies."
                }
            else:
                corpus[doc_id] = {
                    "title": f"Medical Research {i}",
                    "text": f"This study examines treatment efficacy for anxiety disorders, comparing cognitive behavioral therapy (CBT) with pharmacological interventions. Meta-analysis of clinical trials indicates that CBT produces outcomes comparable to medication for many patients, with potentially more durable effects after treatment discontinuation. Combined approaches often yield superior results, particularly for severe or treatment-resistant cases."
                }
    elif current_use_case == "Campus Info":
        # Campus information content
        for i in range(20):
            doc_id = f"doc{i}"
            corpus[doc_id] = {
                "title": f"Missouri S&T Page {i}",
                "text": f"This page contains information about Missouri S&T services and offices. The International Student and Scholar Services (ISSS) office provides support for sponsored international students including visa assistance, cultural activities, and academic guidance. Students can contact the Graduate Studies department for information about graduate programs, thesis requirements, and funding opportunities."
            }

    else:
        # Generic content for any other use case
        for i in range(20):
            doc_id = f"doc{i}"
            corpus[doc_id] = {
                "title": f"Document {i}",
                "text": f"This is sample text for document {i}. It contains information relevant to various queries in this domain. The content includes key facts, explanations, and examples that might be useful for answering questions on this topic."
            }

    return corpus

Download the files if they don't exist


In [ ]:
# --- Path Setup ---
repo_folder_name = selected_dataset.replace('/', '_')
base_path = f"prebuilt_indexes/{repo_folder_name}"
os.makedirs(base_path, exist_ok=True) # Create directory if it doesn't exist

# List of files expected for an index (Keep this)
files_to_download = ["corpus.pkl", "embeddings.npy", "faiss_index.bin", "doc_ids.pkl"] # Add qrels.pkl etc. if needed

print(f"Checking/downloading pre-built indexes for {selected_dataset} from HF Hub: {HUB_REPO_ID}...")

# --- Download Loop (Using HF Hub) ---
all_files_exist = True
for file_name in files_to_download:
    local_file_path = os.path.join(base_path, file_name)
    if not os.path.exists(local_file_path):
        all_files_exist = False # Mark that at least one file needs downloading
        print(f"Downloading {file_name}...")
        try:
            # Construct the path *within* the Hub repository
            # Assumes you uploaded into folders named like 'beir_trec-covid' etc.
            path_in_repo = f"{repo_folder_name}/{file_name}"

            # Use hf_hub_download
            downloaded_path = hf_hub_download(
                repo_id=HUB_REPO_ID,
                filename=path_in_repo,
                repo_type="dataset", # Specify it's a dataset repo
                local_dir=base_path, # Download directly into the target folder
                local_dir_use_symlinks=False # Avoids potential symlink issues
            )
            # Double-check file exists at the expected final path
            if not os.path.exists(local_file_path):
                 if os.path.exists(downloaded_path) and downloaded_path != local_file_path:
                     # Move if hf_hub_download placed it slightly differently (rare with local_dir)
                     os.rename(downloaded_path, local_file_path)
                     print(f"Moved downloaded file to {local_file_path}")
                 else:
                      raise FileNotFoundError(f"Download failed or file not found at expected path {local_file_path} or download path {downloaded_path} for {file_name}")

            print(f"Successfully downloaded {file_name} to {local_file_path}")

        except Exception as e:
            print(f"ERROR downloading {file_name} from Hugging Face Hub: {e}")
            print(f"Check connection and ensure the file exists at 'datasets/{HUB_REPO_ID}/tree/main/{path_in_repo}' on the Hub.")
            # --- Optional: Keep your fallback logic ---
            print(f"⚠️ Creating sample data for {file_name} since download failed")
            # Adapt this fallback logic based on your generate_sample_corpus function and other needs
            if file_name == "corpus.pkl":
                 try:
                     # Ensure generate_sample_corpus and use_case are defined earlier
                     sample_corpus = generate_sample_corpus(selected_dataset, use_case)
                     with open(local_file_path, 'wb') as f: pickle.dump(sample_corpus, f)
                 except NameError:
                     print("generate_sample_corpus or use_case not defined. Cannot create sample data.")
                 except Exception as sample_e:
                     print(f"Error creating sample corpus: {sample_e}")
            elif file_name == "doc_ids.pkl":
                 try:
                     sample_doc_ids = [f"doc{i}" for i in range(20)]
                     with open(local_file_path, 'wb') as f: pickle.dump(sample_doc_ids, f)
                 except Exception as sample_e:
                     print(f"Error creating sample doc_ids: {sample_e}")
            elif file_name == "embeddings.npy":
                 try:
                     sample_embeddings = np.random.rand(20, 384).astype(np.float32)
                     np.save(local_file_path, sample_embeddings)
                 except Exception as sample_e:
                     print(f"Error creating sample embeddings: {sample_e}")
            elif file_name == "faiss_index.bin":
                 try:
                     sample_index = faiss.IndexFlatL2(384); sample_index.add(np.random.rand(20, 384).astype(np.float32))
                     faiss.write_index(sample_index, local_file_path)
                 except Exception as sample_e:
                     print(f"Error creating sample faiss_index: {sample_e}")
            # --- End Optional Fallback ---

# Optional check message
if all_files_exist:
    print("All required index files already exist locally.")

In [ ]:
print(f"Loading pre-built indexes for {selected_dataset}...")

Load corpus

In [ ]:
print("Loading document corpus...")
with open(corpus_path, 'rb') as f:
    corpus = pickle.load(f)

Load document IDs

In [ ]:
print("Loading document IDs...")
with open(doc_ids_path, 'rb') as f:
    doc_ids = pickle.load(f)

Load embeddings

In [ ]:
print("Loading document embeddings...")
doc_embeddings = np.load(embeddings_path)

Load FAISS index

In [ ]:

print("Loading FAISS index...")
index = faiss.read_index(faiss_index_path)

In [ ]:
try:
    print(f"Successfully loaded pre-built indexes for {selected_dataset}")
    print(f"Corpus size: {len(corpus)} documents")
    print(f"Embeddings shape: {doc_embeddings.shape}")
    print(f"FAISS index size: {index.ntotal} vectors")

except Exception as e:
    print(f"Error building indexes: {e}")
    print(f"⚠️ Creating sample data for demonstration")

    # Create sample corpus
    corpus = {}
    for i in range(100):
        doc_id = f"doc{i}"
        corpus[doc_id] = {
            'title': f"Sample Document {i}",
            'text': f"This is sample text for document {i} related to {use_case}. It contains information relevant to various queries in this domain. The content includes key facts, explanations, and examples that might be useful for answering questions on this topic."
        }

    doc_ids = list(corpus.keys())

    # Create sample embeddings
    doc_embeddings = np.random.rand(len(doc_ids), 384).astype(np.float32)
    faiss.normalize_L2(doc_embeddings)

    # Create FAISS index
    index = faiss.IndexFlatIP(doc_embeddings.shape[1])
    index.add(doc_embeddings)

    # Save sample data
    with open(corpus_path, 'wb') as f:
        pickle.dump(corpus, f)

    with open(doc_ids_path, 'wb') as f:
        pickle.dump(doc_ids, f)

    np.save(embeddings_path, doc_embeddings)
    faiss.write_index(index, faiss_index_path)

    print(f"✅ Created sample data with {len(corpus)} documents")


Load queries and qrels from ir_datasets

In [ ]:
import os
import shutil

# Fix the nested directory structure
def fix_nested_directories(dataset_name):
    parent_dir = f"prebuilt_indexes/{dataset_name}"
    nested_dir = f"{parent_dir}/{dataset_name}"

    # Check if the nested structure exists
    if os.path.exists(nested_dir) and os.path.isdir(nested_dir):
        print(f"🔄 Fixing nested directory structure for {dataset_name}...")

        # Move all files from nested dir to parent dir
        for item in os.listdir(nested_dir):
            src = os.path.join(nested_dir, item)
            dst = os.path.join(parent_dir, item)
            if not os.path.exists(dst):  # Avoid overwriting existing files
                shutil.move(src, dst)
                print(f"  ↪ Moved {item} to parent directory")

        # Remove the now-empty nested directory
        try:
            os.rmdir(nested_dir)
            print(f"✅ Removed empty nested directory")
        except OSError:
            print(f"⚠️ Couldn't remove nested directory - it may not be empty")
    else:
        print(f"✓ No nested directory structure found for {dataset_name}")

# Call this function before trying to access the files
fix_nested_directories("custom_mst_site")

In [ ]:
import json
import os

def download_custom_dataset_files(repo_id, dataset_name):
    """
    Download required files for custom dataset from Hugging Face
    """
    base_path = f"prebuilt_indexes/{dataset_name}"
    os.makedirs(base_path, exist_ok=True)

    print(f"📥 Downloading files for {dataset_name} from Hugging Face...")

    # List of files to download
    files_to_download = [
        "topics.json",
        "auto_qrels.txt"
    ]

    downloaded_files = []
    for filename in files_to_download:
        try:
            # Download file from HF
            # Use hf_hub_download

            filepath = hf_hub_download(
                repo_id=HUB_REPO_ID,
                filename=f"{dataset_name}/{filename}",
                repo_type="dataset",
                local_dir=base_path,
                local_dir_use_symlinks=False
            )

            # Copy to our target directory
            target_path = os.path.join(base_path, filename)
            shutil.copy(filepath, target_path)
            downloaded_files.append(filename)
            print(f"  ✅ Downloaded {filename}")
        except Exception as e:
            print(f"  ❌ Failed to download {filename}: {e}")

    return downloaded_files

def load_custom_topics_json(json_path):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    topics_list = data["topics"] if "topics" in data else data
    queries = {}
    for topic in topics_list:
        topic_id = str(topic.get("id") or topic.get("number"))
        queries[topic_id] = {
            "id": topic_id,
            "text": topic.get("title", "").strip(),
            "description": topic.get("description", "").strip(),
            "narrative": topic.get("narrative", "").strip()
        }
    print(f"✅ Loaded {len(queries)} topics from JSON")
    return queries

def load_custom_qrels_txt(qrels_path):
    qrels_dict = {}
    with open(qrels_path, "r", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) == 4:
                topic_id, _, doc_id, relevance = parts
                if topic_id not in qrels_dict:
                    qrels_dict[topic_id] = {}
                qrels_dict[topic_id][doc_id] = int(relevance)
    print(f"✅ Loaded qrels for {len(qrels_dict)} topics")
    return qrels_dict

def select_sample_queries(queries, qrels_dict, max_queries=3):
    sample = {}
    for topic_id, query_info in queries.items():
        if topic_id in qrels_dict:
            sample[topic_id] = query_info
        if len(sample) >= max_queries:
            break
    return sample

try:
    if selected_dataset == "custom_mst_site":
        print("📦 Loading queries and qrels for custom MST dataset...")

        base_path = "prebuilt_indexes/custom_mst_site"
        topics_path = os.path.join(base_path, "topics.json")
        qrels_path = os.path.join(base_path, "auto_qrels.txt")

        # Fix directory structure if needed
        # fix_nested_directories("custom_mst_site")

        # Check if required files exist, download if not
        if not (os.path.exists(topics_path) and os.path.exists(qrels_path)):
            print("🔍 Required files not found locally, downloading from Hugging Face...")
            # Replace "your-hf-repo-id" with the actual Hugging Face repository ID
            download_custom_dataset_files("your-hf-repo-id", "custom_mst_site")

        # Now try to load the files
        queries = load_custom_topics_json(topics_path)
        qrels_dict = load_custom_qrels_txt(qrels_path)
        sample_queries = select_sample_queries(queries, qrels_dict)

        print(f"✅ Selected {len(sample_queries)} sample queries for demonstration")


    else:
        # Fall back to ir_datasets version
        import ir_datasets
        print("📥 Loading queries and relevance judgments from ir_datasets...")
        dataset = ir_datasets.load(selected_dataset)

        queries = {}
        qrels_dict = {}

        try:
            next(dataset.queries_iter())
            for query in dataset.queries_iter():
                query_id = query.query_id if hasattr(query, 'query_id') else getattr(query, '_id', f'q{len(queries)}')
                if hasattr(query, 'text'):
                    query_text = query.text
                elif hasattr(query, 'title'):
                    query_text = query.title
                elif hasattr(query, 'query'):
                    query_text = query.query
                else:
                    query_text = ""
                    for field in dir(query):
                        if not field.startswith('_') and isinstance(getattr(query, field), str) and field != 'query_id':
                            query_text = getattr(query, field)
                            break

                queries[query_id] = {'text': query_text, 'id': query_id}
                if len(queries) >= 100:
                    break
            print(f"✅ Loaded {len(queries)} queries from ir_datasets")
        except:
            print("⚠️ No queries found in ir_datasets")

        try:
            next(dataset.qrels_iter())
            for qrel in dataset.qrels_iter():
                if not hasattr(qrel, 'query_id') or not hasattr(qrel, 'doc_id') or not hasattr(qrel, 'relevance'):
                    continue
                if qrel.query_id not in qrels_dict:
                    qrels_dict[qrel.query_id] = {}
                qrels_dict[qrel.query_id][qrel.doc_id] = qrel.relevance
            print(f"✅ Loaded relevance judgments for {len(qrels_dict)} queries")
        except:
            print("⚠️ No relevance judgments found in ir_datasets")

        sample_queries = {}
        if queries and qrels_dict:
            for query_id, query_info in queries.items():
                if query_id in qrels_dict and len(sample_queries) < 3:
                    sample_queries[query_id] = query_info
        if not sample_queries and queries:
            for i, (query_id, query_info) in enumerate(queries.items()):
                if i < 3:
                    sample_queries[query_id] = query_info

        print(f"✅ Selected {len(sample_queries)} sample queries for demonstration")

except Exception as e:
    print("❌ Error loading queries and qrels:", e)


## Initialize retrieval models

Load bi-encoder model for query encoding

In [ ]:

biencoder_model_name = "sentence-transformers/msmarco-distilbert-base-v3" # @param ["sentence-transformers/msmarco-distilbert-base-v3", "sentence-transformers/all-mpnet-base-v2", "sentence-transformers/all-MiniLM-L6-v2"]
bi_encoder = SentenceTransformer(biencoder_model_name)
bi_encoder.to(device)
print(f"Loaded bi-encoder model: {biencoder_model_name}")

Load cross-encoder model for reranking

In [ ]:

crossencoder_model_name = "cross-encoder/ms-marco-MiniLM-L-6-v2" # @param ["cross-encoder/ms-marco-MiniLM-L-6-v2", "cross-encoder/ms-marco-MiniLM-L-12-v2", "cross-encoder/ms-marco-TinyBERT-L-2-v2"]
cross_encoder = CrossEncoder(crossencoder_model_name)
cross_encoder.to(device)
print(f"Loaded cross-encoder model: {crossencoder_model_name}")

Load LLM for answer generation

In [ ]:
# Set your Mistral API key (preferably from environment variable)
os.environ["MISTRAL_API_KEY"] = "abcdef"  # Replace with your actual key
print("Mistral API configured and ready to use")

## 📏 Domain-Specific Evaluation: Beyond Standard Metrics

Understanding how to evaluate RAG systems properly is critical for real-world applications. Different domains require different evaluation approaches:

### Scientific Research Evaluation
- **Factual Accuracy**: Are all scientific claims supported by the retrieved documents?
- **Citation Quality**: Does the system properly attribute information to sources?
- **Uncertainty Handling**: Does the response appropriately express limitations and caveats?

### Technical Support Evaluation
- **Actionability**: Can a user follow the instructions without additional information?
- **Correctness**: Do the steps actually resolve the technical issue?
- **Safety**: Are proper warnings included for potentially harmful operations?

### Educational Content Evaluation
- **Comprehensibility**: Is the content appropriate for the intended education level?
- **Scaffolding**: Does the explanation build concepts in a logical order?
- **Engagement**: Does the content use appropriate examples and explanations?

### Healthcare Information Evaluation
- **Clinical Accuracy**: Is the medical information correct and up-to-date?
- **Completeness**: Are important warnings, contraindications, or limitations mentioned?
- **Clarity**: Is medical terminology appropriately explained?

### Implementing Custom Evaluation
In this tutorial, we use standard IR metrics (precision, recall, etc.) augmented with domain-specific considerations. In production systems, consider implementing:

1. Human-in-the-loop evaluation pipelines
2. Domain expert review for critical applications
3. Automated checks for domain-specific requirements

Run the next cell to see how we can implement custom evaluation metrics:


In [ ]:
import re

def evaluate_scientific_rag(query, retrieved_docs, generated_answer, ground_truth=None):
    """Evaluate a scientific RAG response with domain-specific metrics"""

    # 1. Check for citations in the answer
    citation_pattern = r'\[(\d+)\]|\(([A-Za-z\s]+,\s*\d{4})\)'
    has_citations = bool(re.search(citation_pattern, generated_answer))

    # 2. Check if answer mentions limitations or uncertainty
    uncertainty_terms = ['may', 'might', 'could', 'possibly', 'suggests',
                         'limited evidence', 'more research', 'not conclusive']
    has_uncertainty = any(term in generated_answer.lower() for term in uncertainty_terms)

    # 3. Check relevance of retrieved documents (simplified)
    # In practice, this might use a trained classifier or human evaluation
    relevance_score = sum(doc['cross_score'] for doc in retrieved_docs) / len(retrieved_docs)

    # 4. Calculate a combined scientific quality score (example only)
    scientific_quality = (
        (2 if has_citations else 0) +
        (1 if has_uncertainty else 0) +
        min(2, relevance_score / 3)  # Scale to 0-2 range
    ) / 5  # Normalize to 0-1

    # Return evaluation results
    return {
        'has_citations': has_citations,
        'acknowledges_uncertainty': has_uncertainty,
        'avg_relevance_score': relevance_score,
        'scientific_quality_score': scientific_quality,
    }

# Example usage with a mock result
example_query = "What is the effectiveness of remdesivir for COVID-19?"
example_docs = [{"cross_score": 7.5}, {"cross_score": 6.8}, {"cross_score": 6.2}]
example_answer = "Studies suggest remdesivir may improve recovery time in some COVID-19 patients, though more research is needed to confirm its effectiveness across different patient populations [1]."

scientific_eval = evaluate_scientific_rag(
    example_query,
    example_docs,
    example_answer
)

print("Scientific Domain Evaluation:")
for metric, value in scientific_eval.items():
    if isinstance(value, bool):
        print(f"- {metric}: {'✅ Yes' if value else '❌ No'}")
    elif isinstance(value, float):
        print(f"- {metric}: {value:.2f}")

### Using Domain-Specific Evaluation in Practice

For each domain, you would develop appropriate evaluation metrics:

1. **Scientific Domain**: Citation practices, uncertainty acknowledgment, evidence quality
2. **Technical Support**: Step clarity, command accuracy, safety considerations
3. **Educational Content**: Grade-level appropriateness, concept scaffolding, engagement
4. **Healthcare Information**: Clinical accuracy, completeness of warnings, clarity

These custom evaluations can be:
- **Automated**: For measurable aspects like presence of citations
- **Semi-automated**: Using classifiers trained on expert-labeled examples
- **Human-in-the-loop**: Expert review for critical applications

In production RAG systems, combining standard IR metrics with these domain-specific evaluations provides a more complete picture of system performance.

In [ ]:
def get_domain_metrics(domain):
    """Return appropriate evaluation metrics for the selected domain."""
    # Base metrics for all domains
    base_metrics = [
        nDCG@5, P@5, R@5, AP
    ]

    # Domain-specific additional metrics
    domain_specific = {
        "Scientific Research": [
            nDCG@10,         # More comprehensive literature review (deeper results)
            R@10,            # Higher recall important for research
            RR               # First relevant result important for fact-checking
        ],
        "Technical Support": [
            RR,              # First relevant result critical for troubleshooting
            P@1, P@3,        # Precision at very top results important
            Rprec            # Balances precision/recall
        ],
        "Education & Library": [
            nDCG@10,         # Students often explore more results
            RBP(p=0.8),      # Models patience of students browsing results
            Judged@10        # Coverage of assessment for educational content
        ],
        "Legal Document Analysis": [
            RBP(p=0.9),      # Very high precision focus (legal requires accuracy)
            nDCG@20,         # May need to review many documents for legal research
            infAP            # Handles incomplete judgments common in legal collections
        ],
        "Healthcare Information": [
            P@1, P@3,        # Top results critical for medical information
            RR,              # First relevant result important for clinical questions
            ERR              # Models utility with diminishing returns
        ],
        "Campus Info": [
            P@1, P@3,        # Precise information location important
            RR,              # First relevant result critical
            nDCG@5           # Overall relevance ranking
        ]
    }

    return base_metrics + domain_specific.get(domain, [])


## Define RAG functions

In [ ]:

def retrieve_documents(query, top_k_first_stage=100, top_k_reranked=5):
    """
    Two-stage retrieval:
    1. Retrieve top_k_first_stage documents using pre-built FAISS index
    2. Rerank with cross-encoder to get final top_k_reranked
    """
    # 1. First-stage retrieval with biencoder + FAISS
    # Encode the query using the bi-encoder model
    query_embedding = bi_encoder.encode(query, show_progress_bar=False, convert_to_numpy=True)
    query_embedding = np.array([query_embedding], dtype=np.float32)
    faiss.normalize_L2(query_embedding)  # Normalize for cosine similarity

    # Search the FAISS index
    scores, indices = index.search(query_embedding, k=min(top_k_first_stage, index.ntotal))

    # 2. Reranking with cross-encoder
    cross_encoder_candidates = []
    for idx in indices[0]:  # indices comes as a 2D array
        doc_id = doc_ids[idx]
        doc_info = corpus[doc_id]
        title = doc_info.get('title', '')
        text = doc_info['text']
        combined_text = f"{title}. {text}" if title else text
        cross_encoder_candidates.append([query, combined_text])

    # Score with cross-encoder
    cross_scores = cross_encoder.predict(cross_encoder_candidates)

    # Sort by cross-encoder scores
    cross_results = []
    for i, idx in enumerate(indices[0]):
        doc_id = doc_ids[idx]
        biencoder_score = float(scores[0][i])  # Convert from numpy float to Python float
        cross_score = float(cross_scores[i])   # Ensure Python float
        cross_results.append((doc_id, biencoder_score, cross_score))

    # Sort by cross-encoder score
    cross_results = sorted(cross_results, key=lambda x: x[2], reverse=True)[:top_k_reranked]

    # Format final results
    results = []
    for doc_id, biencoder_score, cross_score in cross_results:
        doc_info = corpus[doc_id]
        title = doc_info.get('title', '')
        text = doc_info['text']
        combined_text = f"{title}. {text}" if title else text
        results.append({
            'doc_id': doc_id,
            'biencoder_score': biencoder_score,
            'cross_score': cross_score,
            'text': combined_text
        })

    return results

In [ ]:
# Define the Mistral API function for answer generation
def generate_answer(query, context, domain_prompt="", max_length=500):
    """Generate an answer using the Mistral API based on retrieved documents and domain-specific prompting"""

    # You'll need to get an API key from Mistral AI and set it as an environment variable
    # or replace the line below with your actual API key
    api_key = os.environ.get("MISTRAL_API_KEY")
    if not api_key:
        raise ValueError("MISTRAL_API_KEY environment variable not set. Please set it before calling this function.")

    # Mistral API endpoint
    api_url = "https://api.mistral.ai/v1/chat/completions"

    # Create the messages list
    messages = []

    # Add system message with domain-specific prompt if provided
    if domain_prompt:
        messages.append({
            "role": "system",
            "content": domain_prompt.strip()
        })

    # Add user message with context and query
    messages.append({
        "role": "user",
        "content": f"Context:\n{context.strip()}\n\nQuestion: {query.strip()}"
    })

    # Prepare the API request payload
    payload = {
        "model": "mistral-large-latest",  # You can change to other Mistral models as needed
        "messages": messages,
        "max_tokens": max_length,
        "temperature": 0.7,
        "top_p": 0.9,
    }

    # Set up headers with API key
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    try:
        # Make the API request
        response = requests.post(api_url, headers=headers, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the JSON response
        response_data = response.json()

        # Extract the generated text
        generated_text = response_data["choices"][0]["message"]["content"]

        return generated_text

    except Exception as e:
        print(f"Error calling Mistral API: {e}")
        # Fallback to a simple response if API call fails
        return f"I couldn't generate a response using the Mistral API due to an error: {str(e)}"

In [ ]:
def run_rag_pipeline(query, query_id=None, top_k_first_stage=100, top_k_reranked=5, domain_prompt="", show_timings=False):
    """Run the full RAG pipeline with Mistral API for generation"""
    # Timing dictionary
    timings = {}

    # 1. Retrieve and rerank documents
    start_time = time.time()
    retrieved_docs = retrieve_documents(query, top_k_first_stage, top_k_reranked)
    timings['retrieval'] = time.time() - start_time

    # 2. Format context for LLM
    context = ""
    for i, doc in enumerate(retrieved_docs):
        context += f"Document {i+1} [Score: {doc['cross_score']:.3f}]:\n{doc['text']}\n\n"

    # 3. Generate answer with Mistral API
    start_time = time.time()
    answer = generate_answer(query, context, domain_prompt)
    timings['generation'] = time.time() - start_time

    # Total time
    timings['total'] = timings['retrieval'] + timings['generation']

    # Create run (for ir_measures evaluation)
    run = {}
    if query_id is not None:
        run[query_id] = {doc['doc_id']: float(doc['cross_score']) for doc in retrieved_docs}

    return retrieved_docs, answer, timings, run


In [ ]:
def evaluate_with_ir_measures(run, qrels, metrics_list=None):
    """
    Evaluate a run using ir_measures library

    Args:
        run: Dict mapping query_id -> {doc_id -> score}
        qrels: Dict mapping query_id -> {doc_id -> relevance}
        metrics_list: List of ir_measures metric objects (default: common metrics)

    Returns:
        Dict of metric results
    """
    if metrics_list is None:
        # Define default metrics to evaluate
        metrics_list = [
            nDCG@5, nDCG@10,       # Normalized Discounted Cumulative Gain
            P@5, P@10,             # Precision at k
            R@5, R@10,             # Recall at k
            AP,                    # Average Precision
            RR                      # Reciprocal Rank
        ]

    # Calculate aggregate metrics
    try:
        results = ir_measures.calc_aggregate(metrics_list, qrels, run)
        return results
    except Exception as e:
        print(f"Error during evaluation: {e}")
        return {}

In [ ]:
from IPython.display import display, HTML, Markdown

def display_rag_results(query, retrieved_docs, answer, timings=None, metrics=None, use_case=None):
    """Display RAG results in a formatted way with domain-specific formatting"""

    display(HTML(f"<h2 style='color:#eee;'>RAG Results for {use_case if use_case else 'Query'}</h2>"))
    display(HTML(f"<h3 style='color:#ccc;'>📝 Query: {query}</h3>"))

    # Display metrics if available
    if metrics:
        display(HTML("<h3 style='color:#ccc;'>📊 Retrieval Metrics:</h3>"))
        metrics_html = "<table style='color:#ddd; border-collapse: collapse;'>"
        metrics_html += "<tr><th style='padding: 4px 10px;'>Metric</th><th style='padding: 4px 10px;'>Value</th></tr>"
        for metric_name, metric_value in metrics.items():
            metrics_html += f"<tr><td style='padding: 4px 10px;'>{metric_name}</td><td style='padding: 4px 10px;'>{metric_value:.4f}</td></tr>"
        metrics_html += "</table>"
        display(HTML(metrics_html))

    # Display retrieved documents
    display(HTML("<h3 style='color:#ccc;'>📚 Retrieved Documents:</h3>"))
    for i, doc in enumerate(retrieved_docs):
        display(Markdown(f"**Document {i+1}:**"))
        display(Markdown(f"- **Biencoder Score:** {doc['biencoder_score']:.3f}"))
        display(Markdown(f"- **Cross-Encoder Score:** {doc['cross_score']:.3f}"))
        display(Markdown(f"- **Document ID:** {doc['doc_id']}"))
        display(Markdown(f"- **Text:** {doc['text'][:300]}..."))
        print()

    # Display generated answer
    display(HTML("<h3 style='color:#ccc;'>🤖 Generated Answer:</h3>"))

    # Apply consistent dark-mode-friendly, domain-specific styling
    color_map = {
        "Scientific Research": "#3498db",
        "Technical Support": "#2ecc71",
        "Education & Library": "#f39c12",
        "Legal Document Analysis": "#9b59b6",
        "Healthcare Information": "#e74c3c",
        "Campus Info": "#16a085"
    }
    border_color = color_map.get(use_case, "#7f8c8d")  # default gray

    answer_html = f'''
    <div style="
        border-left: 5px solid {border_color};
        padding: 12px 16px;
        margin: 10px 0;
        background-color: #1e1e1e;
        color: #e0e0e0;
        border-radius: 8px;
        font-size: 15px;
        line-height: 1.6;">
        {answer}
    </div>
    '''

    display(HTML(answer_html))

    # Display timings if available
    if timings:
        display(HTML("<h3 style='color:#ccc;'>⏱️ Performance Metrics:</h3>"))
        timings_html = "<table style='color:#ddd; border-collapse: collapse;'>"
        timings_html += "<tr><th style='padding: 4px 10px;'>Metric</th><th style='padding: 4px 10px;'>Time (seconds)</th></tr>"
        timings_html += f"<tr><td style='padding: 4px 10px;'>Retrieval time</td><td style='padding: 4px 10px;'>{timings['retrieval']:.3f}</td></tr>"
        timings_html += f"<tr><td style='padding: 4px 10px;'>Generation time</td><td style='padding: 4px 10px;'>{timings['generation']:.3f}</td></tr>"
        timings_html += f"<tr><td style='padding: 4px 10px;'>Total RAG time</td><td style='padding: 4px 10px;'>{timings['total']:.3f}</td></tr>"
        timings_html += "</table>"
        display(HTML(timings_html))


## Run RAG on Sample Queries for the Selected Use Case

In [ ]:
print(f"\n=== Sample Queries from {use_case} Use Case ===\n")

Get domain-specific prompt

In [ ]:
domain_prompt = use_cases[use_case]["domain_prompt"]
domain_metrics = get_domain_metrics(use_case)

In [ ]:
print(domain_prompt)
print(domain_metrics)

Store results for each sample query

In [ ]:
sample_results = []
combined_run = {}  # For evaluation across all queries

In [ ]:
for query_id, query_info in sample_queries.items():
    query_text = query_info['text']
    print(f"Running RAG for query: '{query_text}'")

    # Run the RAG pipeline with domain-specific prompt
    retrieved_docs, answer, timings, run = run_rag_pipeline(
        query_text,
        query_id=query_id,
        top_k_first_stage=100,
        top_k_reranked=5,
        domain_prompt=domain_prompt
    )

    # Combine run for overall evaluation
    combined_run.update(run)

    # Store results
    sample_results.append({
        'query_id': query_id,
        'query_text': query_text,
        'retrieved_docs': retrieved_docs,
        'answer': answer,
        'timings': timings,
        'run': run
    })

    # Display a brief summary
    print(f"- Retrieved {len(retrieved_docs)} documents")
    print(f"- Answer length: {len(answer)} characters")
    print("-----------------------------------")

Evaluate all sample queries using `ir_measures` with domain-specific metrics

In [ ]:
if combined_run and qrels_dict:
    print("\n=== Evaluation with ir_measures ===\n")

    # Calculate metrics
    metrics_results = evaluate_with_ir_measures(combined_run, qrels_dict, domain_metrics)

    # Display results
    print("Overall Evaluation Results:")
    for metric, value in metrics_results.items():
        print(f"  - {metric}: {value:.4f}")

Detailed Results Viewer with Domain-Specific Visualization

In [ ]:
query_index = 1  # @param ["0", "1", "2"] {type:"raw"}
show_timings = True # @param {type:"boolean"}

In [ ]:
if sample_results and query_index < len(sample_results):
    result = sample_results[query_index]

    # Evaluate this specific query with ir_measures using domain-specific metrics
    if qrels_dict and result['query_id'] in qrels_dict:
        metrics = evaluate_with_ir_measures(
            result['run'],
            qrels_dict,
            domain_metrics
        )
    else:
        metrics = None

    # Display results with domain-specific formatting
    display_rag_results(
        result['query_text'],
        result['retrieved_docs'],
        result['answer'],
        result['timings'] if show_timings else None,
        metrics,
        use_case
    )

## Try RAG with Your Own Query for this Domain

In [ ]:

user_query = "What are the current research focus areas Sajal Das of Missouri S&T's Computer Science department faculty, and which projects have received major funding in the last year?" # @param {type:"string"}
top_k_first_stage = 100 # @param {type:"slider", min:10, max:500, step:10}
top_k_reranked = 5 # @param {type:"slider", min:1, max:20, step:1}
show_timings = True # @param {type:"boolean"}

In [ ]:
# If no query is provided, use a default example from the selected use case
if not user_query:
    user_query = use_cases[use_case]["example_queries"][0]
    print(f"Using example query: \"{user_query}\"")
    print("Enter your own query above to try something different!")
else:
    print(f"Processing custom query: \"{user_query}\"")

In [ ]:
# Get domain-specific prompt
domain_prompt = use_cases[use_case]["domain_prompt"]

In [ ]:
# Run the RAG pipeline with domain-specific prompt
retrieved_docs, answer, timings, _ = run_rag_pipeline(
    user_query,
    top_k_first_stage=top_k_first_stage,
    top_k_reranked=top_k_reranked,
    domain_prompt=domain_prompt,
    show_timings=show_timings
)

In [ ]:
# Display results with domain-specific formatting
display_rag_results(
    user_query,
    retrieved_docs,
    answer,
    timings if show_timings else None,
    use_case=use_case
)

### 📊 Understanding the Results

Let's analyze what happened in this RAG example:

1. **Retrieval Quality**:
   - Examine the relevance scores of retrieved documents
   - Higher cross-encoder scores indicate better matches to the query
   - Does the first document contain the information needed?

2. **Answer Completeness**:
   - Check if the generated answer incorporates information from the retrieved documents
   - Is the answer comprehensive or missing key information?

3. **Domain Appropriateness**:
   - Does the response match the expected format for this domain?
   - For scientific content: Are citations and caveats included?
   - For technical support: Are steps clear and actionable?
   - For educational content: Is the explanation accessible and structured?

Try modifying the query slightly and observe how the retrieval results change.

## Demonstrating LLM Hallucination Without Retrieval

To illustrate why the retrieval component is crucial in RAG systems, let's compare our previous RAG results with what happens when we ask the same question directly to the LLM without providing retrieved documents.

In [ ]:
print("🔍 With RAG vs. Without RAG Comparison")
print(f"\nQuery: \"{user_query}\"")

# Function to generate answer without retrieval context
def generate_answer_without_retrieval(query, use_case="Scientific Research", max_length=500):
    """Generate an answer using the Mistral API without any retrieved context"""

    api_key = os.environ.get("MISTRAL_API_KEY")
    if not api_key:
        raise ValueError("MISTRAL_API_KEY environment variable not set. Please set it before calling this function.")

    # Mistral API endpoint
    api_url = "https://api.mistral.ai/v1/chat/completions"

    # Create domain-specific prompts without retrieval references
    no_retrieval_prompts = {
        "Scientific Research": "You are a scientific research assistant. Provide an accurate and evidence-based answer to the user's question. Include appropriate scientific context and caveats where applicable.",

        "Technical Support": "You are a technical support specialist. Provide a concise and practical solution to the user's problem. Structure your answer step-by-step, and ensure instructions are clear and executable.",

        "Education & Library": "You are an educational assistant. Provide a well-structured, accurate, and learner-friendly explanation of the topic. Break down complex concepts into simpler terms and include definitions or examples when needed.",

        "Fact Verification": "You are a fact-checking assistant. Provide a balanced, evidence-based analysis of the user's query. If information is inconclusive or uncertain, clearly state the limitations in current knowledge.",

        "Healthcare Information": "You are a healthcare information assistant. Provide an accurate summary addressing the user's question. Note that this is not medical advice. Focus on established research findings and clinical guidelines, and clearly state any limitations in the available evidence.",

        "Campus Info": "You are a helpful campus assistant for Missouri S&T. Answer the user's question clearly and accurately. Be concise and maintain a friendly and professional tone."
    }

    # Get the prompt based on the use case
    no_retrieval_prompt = no_retrieval_prompts.get(use_case, "You are a helpful assistant. Answer the user's question accurately and concisely.")

    # Create the messages list
    messages = []

    # Add system message with the no-retrieval prompt
    messages.append({
        "role": "system",
        "content": no_retrieval_prompt.strip()
    })

    # Add user message with only the query
    messages.append({
        "role": "user",
        "content": query.strip()
    })

    # Prepare the API request payload
    payload = {
        "model": "mistral-large-latest",  # You can change to other Mistral models as needed
        "messages": messages,
        "max_tokens": max_length,
        "temperature": 0.7,
        "top_p": 0.9,
    }

    # Set up headers with API key
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }

    try:
        # Make the API request
        response = requests.post(api_url, headers=headers, json=payload)
        response.raise_for_status()  # Raise an exception for HTTP errors

        # Parse the JSON response
        response_data = response.json()

        # Extract the generated text
        generated_text = response_data["choices"][0]["message"]["content"]

        return generated_text

    except Exception as e:
        print(f"Error calling Mistral API: {e}")
        # Fallback to a simple response if API call fails
        return f"I couldn't generate a response using the Mistral API due to an error: {str(e)}"

In [ ]:
# Start timing
start_time = time.time()

# Generate answer without retrieval using the current use case
no_retrieval_answer = generate_answer_without_retrieval(user_query, use_case)
no_retrieval_time = time.time() - start_time

# Display results
from IPython.display import display, HTML, Markdown

display(HTML("<h2 style='color:#eee;'>Comparing RAG vs. No Retrieval</h2>"))

# Display the comparison in a side-by-side view
comparison_html = f'''
<div style="display: flex; flex-direction: row; gap: 20px;">
    <div style="flex: 1; border-radius: 8px; padding: 16px; background-color: #1e1e1e;">
        <h3 style="color: #3498db;">✅ With Retrieval (RAG)</h3>
        <p style="color: #aaa; font-style: italic;">Response time: {timings["total"]:.3f}s</p>
        <div style="border-left: 5px solid #2ecc71; padding: 12px; background-color: #2a2a2a; color: #e0e0e0; border-radius: 4px;">
            {answer}
        </div>
    </div>
    <div style="flex: 1; border-radius: 8px; padding: 16px; background-color: #1e1e1e;">
        <h3 style="color: #e74c3c;">❌ Without Retrieval</h3>
        <p style="color: #aaa; font-style: italic;">Response time: {no_retrieval_time:.3f}s</p>
        <div style="border-left: 5px solid #e74c3c; padding: 12px; background-color: #2a2a2a; color: #e0e0e0; border-radius: 4px;">
            {no_retrieval_answer}
        </div>
    </div>
</div>
'''

display(HTML(comparison_html))

### 🔍 Understanding Hallucinations in LLMs

In the comparison above, we can observe how the LLM's response differs with and without retrieval:

1. **Without Retrieval**:
   - The model generates a plausible-sounding answer based on its training data
   - It may include outdated, incomplete, or incorrect information
   - There is no way to verify claims or trace information sources
   - The model might confidently state incorrect facts

2. **With Retrieval (RAG)**:
   - The response is grounded in specific documents
   - Information is traceable to sources
   - The model can acknowledge uncertainty when evidence is limited
   - The answer quality is limited by the quality of the retrieved documents

This demonstrates why RAG is essential for applications requiring factual accuracy, especially in domains like healthcare, scientific research, technical support, and education.

When evaluating RAG systems, it's valuable to compare with non-RAG baselines to measure the improvement in factual accuracy and the reduction in hallucinations.


## Best Practices for RAG in Each Domain

### Scientific Research
- Use domain-specific embeddings trained on scientific literature.
- Implement citation tracking to show provenance of information.
- Consider combining with a knowledge graph for concept relationships.
- Include publication date as a ranking factor for recency.
- Use subject-specific vocabularies for query expansion.

### Technical Support
- Optimize for high precision in top results.
- Implement conversational context to track troubleshooting steps.
- Use step detection to format answers as procedures.
- Consider hybrid retrieval combining semantic and keyword search.
- Add feedback mechanisms to improve answer quality over time.

### Education & Library
- Organize retrieved content by difficulty/education level.
- Implement learning path generation based on prerequisite concepts.
- Support multiple learning styles in answer generation.
- Include multimedia content recommendations when available.
- Use readability scores to match content to learner level.

### Legal Document Analysis
- Prioritize precision and traceability of information.
- Implement jurisdiction detection for relevant legal standards.
- Track legal precedent relationships between documents.
- Use specialized legal embeddings trained on case law.
- Include confidence scores and disclaimers in generated answers.

### Healthcare Information
- Implement evidence quality assessment for medical information.
- Include recency as a critical ranking factor.
- Use medical taxonomy mapping for query expansion.
- Apply stricter fact verification for medical claims.
- Include appropriate medical disclaimers in answers.

### Campus Info
- Use location-aware retrieval for campus service queries.
- Implement up-to-date checking for office hours and contact info.
- Include department hierarchy awareness for better routing.
- Use hybrid search combining exact name matching with semantic.
- Prioritize official pages over event announcements.

---

## General Implementation Guidance for RAG

When implementing RAG for a specific domain, also consider:

- **Pre-processing:** Apply domain-specific cleaning and normalization.
- **Chunking strategy:** Adjust document chunking based on domain document structure.
- **Embedding models:** Fine-tune or select embeddings trained on domain data.
- **LLM prompting:** Craft specialized prompts with domain context.
- **Post-processing:** Format answers according to domain conventions.

If you do not have a domain-specific use case:

- Select appropriate embedding models for your specific use case.
- Experiment with chunking strategies based on your document structure.
- Fine-tune retrieval parameters (k values, reranking) based on evaluation.
- Craft effective prompts that include relevant context for the LLM.
- Implement user feedback mechanisms to improve over time.


## 🛠️ Troubleshooting Common Issues

When implementing RAG systems, you may encounter these common challenges:

### Retrieval Problems

1. **Irrelevant Documents Retrieved**
   - **Symptoms**: Retrieved documents don't match query intent
   - **Solutions**:
     - Try different embedding models (domain-specific if available)
     - Adjust chunking strategy (smaller or larger chunks)
     - Implement query expansion or reformulation
     - Add a reranking step with a cross-encoder

2. **Missing Information**
   - **Symptoms**: System can't find information you know exists in the corpus
   - **Solutions**:
     - Increase the number of retrieved documents (k)
     - Try hybrid retrieval (combine dense and sparse methods)
     - Check document preprocessing (ensure no information loss)

### Generation Issues

1. **Hallucinations Despite RAG**
   - **Symptoms**: Model generates incorrect information despite relevant context
   - **Solutions**:
     - Adjust prompt to emphasize using only retrieved information
     - Implement fact-checking or confidence scoring
     - Use a model with better instruction following

2. **Poor Integration of Retrieved Content**
   - **Symptoms**: Answer doesn't incorporate relevant retrieved information
   - **Solutions**:
     - Improve context formatting in the prompt
     - Try different LLMs or parameter settings
     - Test chain-of-thought or step-by-step reasoning

### API and Resource Issues

1. **Slow Performance**
   - Try batch processing where possible
   - Consider quantized models or optimized libraries
   - Use caching for repeated queries

2. **API Failures**
   - Implement robust error handling and retries
   - Have fallback models or approaches ready
   - Cache results whenever possible

If you encounter any of these issues during the tutorial, raise your hand, and we'll work through them together.

## Evaluation Summary for Current Use Case

## 🎓 Conclusion: Building Effective Domain-Specific RAG Systems

This tutorial has demonstrated how to build and customize RAG systems for different domains using prebuilt indexes from the `ir_datasets` library. By leveraging public datasets and sentence-transformer embeddings, we've implemented practical RAG solutions tailored to real-world use cases.

### Core Principles for Domain-Specific RAG

1. **Know Your Domain**
   - Understand the specific information needs and constraints
   - Identify domain-specific language, terminology, and concepts
   - Recognize the appropriate evidence standards and response formats

2. **Customize Each Component**
   - Data sources: Select and preprocess domain-appropriate content
   - Embedding models: Choose or fine-tune for domain language
   - Retrieval strategy: Optimize parameters for domain needs
   - Prompting: Design domain-specific instructions and constraints
   - Evaluation: Measure what matters for your specific use case

3. **Iterate with Feedback**
   - Collect domain expert evaluation on system outputs
   - Analyze failure cases to identify improvement areas
   - Continuously update knowledge bases as domains evolve

### Why Domain-Specific RAG Matters

Adapting RAG systems to specific domains significantly improves performance:

- **Specialized Knowledge**: Retrieve content that aligns with domain-specific needs
- **Improved Contextual Understanding**: Domain-aware prompting leads to higher-quality answers
- **Targeted Evaluation**: Metrics tailored to domain objectives provide realistic assessments
- **Enhanced User Experience**: Formatting and output are better suited to user expectations
- **Higher Relevance**: Focused retrieval leads to better precision and recall

### From Tutorial to Production

To move from this tutorial to production-ready systems:

1. **Scale Your Data Pipeline**
   - Implement efficient document ingestion workflows
   - Set up regular knowledge base updates
   - Consider hybrid storage solutions for different content types

2. **Optimize for Performance**
   - Benchmark and optimize vector search for your scale
   - Implement caching strategies for common queries
   - Consider quantization for embedding models

3. **Enhance with Advanced Techniques**
   - Query rewriting and decomposition
   - Multi-stage retrieval architectures
   - Ensemble methods for improved accuracy
   - Hybrid search combining sparse and dense methods
   - Sophisticated chunking strategies (overlapping, hierarchical, semantic)
   - Self-reflection and validation capabilities

4. **Implement Robust Monitoring**
   - Track retrieval quality metrics over time
   - Monitor for knowledge gaps and hallucinations
   - Implement feedback loops from end users

### Suggested Next Steps

To expand upon this tutorial:

- **Build Your Own Indexes**:
  - Use datasets from `ir_datasets` such as `beir/trec-covid`, `nfcorpus`, `cqadupstack`
  - Generate dense embeddings using Sentence Transformers
  - Build and store FAISS indexes for fast retrieval

- **Customize for Your Domain**:
  - Create tailored prompt templates
  - Design custom evaluation methods
  - Use visualizations relevant to your domain
  - Implement hybrid retrieval suited to the task

- **Explore Advanced RAG Architectures**:
  - Experiment with multi-hop RAG for complex reasoning
  - Try Hypothetical Document Embeddings (HyDE)
  - Integrate knowledge graphs with vector search
  - Implement fine-tuning for domain-specific retrievers

### Further Learning Resources

- **[ir_datasets](https://ir-datasets.com/)** & [GitHub](https://github.com/allenai/ir_datasets)
- **[BEIR Benchmark](https://github.com/beir-cellar/beir)**
- **[HuggingFace Datasets](https://huggingface.co/datasets?search=irds)**
- **[Sentence Transformers](https://www.sbert.net/)**
- **[FAISS Library](https://github.com/facebookresearch/faiss)**
- **[LangChain Docs](https://python.langchain.com/docs/modules/data_connection/retrievers/)**
- **[LlamaIndex Docs](https://docs.llamaindex.ai/)**

The field of RAG is rapidly evolving, with new techniques and models emerging regularly. By focusing on domain-specific adaptations, you can build systems that deliver truly valuable and trustworthy information experiences across scientific research, technical support, education, fact verification, healthcare, and beyond.


## 🧭 A Note on Real-World RAG: Handling Multiple Data Sources with Query Routing

In this tutorial, we’ve simplified the setup by letting you choose a single **use case** (e.g., *Scientific Research*, *Technical Support*), and loading **one pre-built index** corresponding to a default dataset for that domain (like `beir/trec-covid` or `beir/cqadupstack/android`). All queries are executed against this single, focused index.

This works well for learning the core components of Retrieval-Augmented Generation (RAG) in a controlled environment. However, in production systems, the reality is far more complex.

---

### 🔍 Real-World RAG Involves Many Diverse Sources

Imagine building a RAG system for an enterprise or large-scale application. A single unified index often isn’t practical or efficient. Real-world systems typically need to answer questions using content drawn from multiple **heterogeneous sources**, such as:

- ✅ Internal technical documentation (e.g., Confluence, GitHub wikis)  
- ✅ Public product FAQs and marketing sites  
- ✅ A database of customer support tickets or live chat logs  
- ✅ Regulatory PDFs, whitepapers, or clinical guidelines  
- ✅ Web content or external APIs

Indexing all of this into one giant vector store is usually **not scalable**, and can dilute retrieval precision. Instead, most real-world systems maintain **multiple specialized indexes**, each optimized for a different content type, domain, or access policy.

---

### 🚦 Enter Query Routing

So how does a system decide **which index (or tool) to query** for a given user question?

This is where **Query Routing** becomes essential. A *Query Router* acts like a smart traffic controller inside the RAG pipeline. Before retrieving documents, it analyzes the incoming query and chooses the most relevant source(s) — or even routes the request to external tools like search APIs, databases, or reasoning modules.

---

### 🧠 Common Query Routing Strategies

Here are several routing strategies used in modern RAG systems:

1. **LLM-Based Routing**  
   Use a lightweight [language model](https://arxiv.org/abs/2303.11366) to classify or analyze the query intent. The LLM chooses the best index by comparing the query to high-level descriptions of each source.  
   *Example:* “What is the refund policy for software licenses?” → routed to the policy documents index.

2. **Semantic Routing (Embedding Similarity)**  
   Compute a dense vector (embedding) of the query, then compare it to “representative vectors” for each index. This can be done using [FAISS](https://github.com/facebookresearch/faiss), [ScaNN](https://github.com/google-research/google-research/tree/master/scann), or [milvus](https://milvus.io/).  
   *Example:* Map queries to closest-matching domains using cosine similarity.

3. **Keyword or Rule-Based Routing**  
   Set up simple keyword triggers or regex patterns to map queries to sources.  
   *Example:* Queries containing “error code” or “stack trace” → routed to technical docs.  
   This is fast and interpretable, but brittle if query language varies a lot.

4. **Metadata Filtering within Shared Indexes**  
   If you choose to merge multiple datasets into one index, you can still route using metadata fields (e.g., `document_source = 'faq'`). This lets you use **filters** to scope retrieval only to relevant subsets.

---

### 🏗️ Hybrid Routing Is Common

In practice, many systems combine these approaches:

- Use **keyword filtering** for high-precision rules  
- Fall back to **semantic routing** for open-ended questions  
- Use an **LLM router** for nuanced intent classification  
- Combine results from multiple indexes and rerank with a **cross-encoder** or **relevance model**

Frameworks like [LangChain](https://docs.langchain.com/docs/components/retrievers/router-retriever/) and [Haystack](https://docs.haystack.deepset.ai/docs/routing_queries) offer modules for building query routers out-of-the-box.

---

### 🛠️ Why We Didn’t Use Query Routing *Here*

For this tutorial, we kept the setup simple by pre-selecting one dataset per domain and routing all queries to a single index. This helps focus on learning RAG fundamentals like retrieval, reranking, and generation — without worrying about architectural complexity.

But in real-world applications — especially in enterprise, healthcare, legal, or customer support scenarios — **query routing is essential** to handle:
- Information silos
- Data volume
- Index-specific latency
- Access control

If you're building a production-ready RAG system, consider implementing query routing as an early design decision.

---

### 📚 Further Reading & Examples

- [Query Routing in LangChain](https://docs.langchain.com/docs/components/retrievers/router-retriever/)
- [Multihop RAG Routing in Google's FiD](https://arxiv.org/abs/2007.00849)
- [Retrieval Strategies in Haystack](https://docs.haystack.deepset.ai/docs/routing_queries)
- [Prompting for Tool Use](https://arxiv.org/abs/2302.12337) in Multi-Tool LLM systems